In [9]:
import os
import glob
import json
import random

import cv2
import matplotlib.pyplot as plt

from tqdm.auto import tqdm

files = glob.glob("data/train/images/*.jpg")
print(f"Found {len(files)} files")

labelid = 0
dataset = []
for file in tqdm(files):
    label_file = file.replace("images", "labels").replace(".jpg", ".txt")

    image = cv2.imread(file)
    height, width, label_id = image.shape

    with open(label_file, "r", encoding="utf-8") as f:
        lines = f.readlines()

    objects = {
        "id": [],
        "area": [],
        "bbox": [],
        "category": [],
    }
    for line in lines:
        parts = line.strip().split()
        if len(parts) != 5:
            continue
        class_id, x_center, y_center, bbox_width, bbox_height = map(float, parts)
        class_id = int(class_id)

        x_center = int(x_center* width)
        y_center = int(y_center * height)
        bbox_width = int(bbox_width * width)
        bbox_height = int(bbox_height * height)

        x_min = x_center - bbox_width // 2
        y_min = y_center - bbox_height // 2

        area = bbox_width * bbox_height
        bbox = [x_min, y_min, bbox_width, bbox_height]

        objects["area"].append(area)
        objects["bbox"].append(bbox)
        objects["category"].append(class_id)

        objects["id"].append(label_id)
        label_id += 1


    dataset.append({
        "image_id": os.path.basename(file).split(".")[0],
        "image": file.replace("\\", "/"),
        "width": width,
        "height": height,
        "objects": objects,
    })


with open("data/train.json", "w", encoding="utf-8") as f:
    json.dump(dataset, f, indent=4, ensure_ascii=False)

Found 30338 files


  0%|          | 0/30338 [00:00<?, ?it/s]